# Overview
This notebook explores different objectives and different constraint enforcement techniques for PSO.

In [11]:
include("../../src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, Constraints, PSO, Objectives, Designs
using Base.Iterators
using BenchmarkTools

In [12]:
function run_experiments(objectives, enforcers, save_func)
    for (obj, enf) in Iterators.product(objectives, enforcers)
        println("Objective: $obj, Enforcer: $enf")
        context = PSO.create_context(
            experiment, 
            obj; 
            callback = PSO.aggregate_results(;save_world=true), 
            runner_params=runner_params,
            enforcer_type = enf
        )
        runner_state, history = PSO.optimize(context)
        save_func(history, obj, enf)
        println("Best score: $(runner_state.swarm.memory.global_best_score)")
    end
end

function save_3d_results(; prefix = "")
    return (history, obj, enf) -> PSO.save_history_3d(history; location = "$(prefix)_$(obj)_$(enf).npy")
end

save_3d_results (generic function with 1 method)

In [13]:
N = 200
K = 50
bound = 100

A, b = Constraints.hypercube(K, bound)
experiment = Experiments.create(N, K)
experiment = Experiments.with_linear_constraints(experiment, A, b)
for i in 1:(K-1)
    experiment = Experiments.with_factor_ratio(experiment, i, K, 2.0)
end

In [ ]:
runner_params = PSO.runner_params(200, 200, 1e-6)
pso_params = PSO.create_hyperparams(100)

objectives = [Objectives.ackley]
# enforcers = [ConstraintEnforcement.Penalty, ConstraintEnforcement.Parametric, ConstraintEnforcement.Resample]
# objectives = [Objectives.griewank]
enforcers = [ConstraintEnforcement.Penalty]
run_experiments(objectives, enforcers, save_3d_results(prefix="../data/10kd_unconstrained"))

Objective: ackley, Enforcer: Penalty
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
149 rows, 51 cols, 347 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-149); columns 0(-51); elements 0(-347) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  

: 